## Assignment 1: MongoDB

This assignment is based on content discussed in Module 2: Introduction to MongoDB

## Learning outcomes 

The purpose of this assignment is for learners to be able to:
- Familarize with JSON document syntax
- Understand basic MongoDB CRUD operations
- Understand MongoDB data pipelines to run aggregate queries

In this assignment, you will make use of the sample data provided in Module 2.  

This dataset has 3 collections: Employee, Workplace and Address.  You will import this data into your local MongoDB database.

Required imports for this project are given below. Make sure you have all libraries required for this project installed. You may use conda or pip based on your set up.

## Setup Notes

**Please note** These instructions are duplicated here for your reference. You would have encountered them originally in Module 2 Part 2. It is expected that you have already taken the steps to set up and run MongoDB.

We will be using MongoDB Community Edition.  The MongoDB database <b>MUST</b> be installed and running locally before continuing with this notebook.  We will need to install two packages using the Anaconda package manager:

1. [Mongodb](https://www.mongodb.com/) - this package contains the mongodb database 
2. [PyMongo](http://api.mongodb.com/python/current/) - this packages contains the python driver that will allow us to communicate with the mongodb database.

#### Install

1. Open a command line terminal and execute the following commad to install mongodb.
```console
conda install -c anaconda mongodb
```
2. Open a command line terminal and execute the following commad to install pymongo packge.
```console
conda install -c anaconda pymongo
```

#### Run Mongodb in Windows
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is \data\db. Create this folder using the following commands from a Command Prompt:
```console
md \data\db
```

2. To start MongoDB, run mongod.exe. For example, from the Command Prompt:
```console
"C:\Program Files\MongoDB\Server\3.2\bin\mongod.exe"
```

#### Run Mongodb in Mac
1. MongoDB requires a data directory to store all data. MongoDB’s default data directory path is /data/db. Create this folder using the following commands from a Command Prompt.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mkdir /data/db/
```

2. To start MongoDB, run mongod.exe. For example, from the Command Promp.  Note that we run the command as a super user using the "sudo" command:
```console
sudo mongod --config /usr/local/etc/mongod.conf
```

In [1]:
#required imports
import os
import json
import datetime
import pymongo
import pprint
import pandas as pd
import numpy as np
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)

Mongo version 3.12.0


We first need to connect to our locally running MongoDB database (<b>Make sure your database is running on your machine</b>). We will use the MongoClient to connect to a local 'test' database that is running on port 27017 (this is the default port).

In [31]:
client = MongoClient('localhost', 27017)
db = client.assignment1

After installing necessary modules proceed to import the data into your database.

In [3]:
# Let's delete any existing collections in our database
db.workplace.drop()
db.address.drop()
db.employee.drop()

# Import our files into our three collections
with open('Data/Employee.json') as f:
    db.employee.insert_many(json.load(f))
with open('Data/Workplace.json') as f:
    db.workplace.insert_many(json.load(f))
with open('Data/Address.json') as f:
    db.address.insert_many(json.load(f))

#### Question 1 (10 Marks)

The address collection contains employee from different ages and interests.  Perform a simple query to list all employees that are less than or equal to 50 and like Cooking.

__NOTE:__ the following shows the structure of an Employee document that will help you construct the query.

In [4]:
pprint.pprint(client.assignment1.employee.find_one())

{'_id': '9f39da36-82cc-4353-ab90-d616105fa7c1',
 'address_id': 'b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8',
 'age': 40,
 'email': 'ih@ri.ro',
 'firstname': 'Emilie',
 'interests': ['Bowling', 'Cooking', 'Golf', 'Swimming'],
 'lastname': 'Woods',
 'workplace_id': 'a32bf18d-e0e5-48f2-a851-aa49c80f9460'}


In [34]:
# In this first option I just looked for the match in age and interests array.
cursor = db.employee.find({"interests": "Cooking", "age": {"$lte": 50}})
pd.DataFrame(list(cursor))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,9f39da36-82cc-4353-ab90-d616105fa7c1,Emilie,Woods,40,ih@ri.ro,"[Bowling, Cooking, Golf, Swimming]",b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,af27265e-6639-49f2-991e-193275a4111a,Thomas,Patterson,18,sug@gon.bf,"[Cooking, Cricket, Tennis, Swimming, Fishing]",64fd714d-e219-4e45-888b-cc2238a8bd0b,5345fcb9-6297-4b9f-aa15-cbee8460f28f
2,00289d48-bad8-4b73-a359-a1a1f05c96e2,Sophia,Flores,22,ra@dupnejuk.nr,"[Hiking, Soccer, Bowling, Rubgy, Cooking, Danc...",8a430805-00b8-40a6-bd93-c950b544a83b,b12cd444-e65b-4bc2-8cf6-2dbe854a627b
3,da76e52b-b3db-4fc0-b0d6-435d1aed0cd9,Ollie,Barnett,25,ro@nemaw.et,"[Cooking, Bowling, Dancing]",5d3eacc4-d1d8-459b-973c-3bc71feacf50,cb795afb-8dc3-482f-b3a0-14229a280afa
4,51643cd6-49bb-45d5-bd6e-717c62bb2869,James,Wilkins,27,hutfardu@vicbiri.gb,"[Rubgy, Tennis, Cricket, Cooking]",3f10d9f7-57ef-40e1-a97d-d2ee53aa2c6e,cb795afb-8dc3-482f-b3a0-14229a280afa
5,f073a705-6546-4375-adb5-b224871776ef,Aaron,Carr,25,fekegim@lucul.tp,[Cooking],f4e59d7e-ea40-442d-87be-106e3c46a554,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
6,457ef68c-9651-4925-bca0-15e246661d19,Alta,Sharp,34,jus@goal.bn,"[Cricket, Cycling, Rubgy, Golf, Cooking, Dancing]",5e97658e-5809-41b8-a088-eddbd81f86a7,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
7,840184a3-4c4d-4b15-8813-30fca6e7827b,Delia,Douglas,36,me@wak.ne,"[Cricket, Cooking, Hiking, Dancing, Tennis]",456e18ae-c2f8-443e-899d-f2b893499695,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
8,6157dc3b-ee2d-463a-b62b-1cd4eed7d575,Dominic,Wade,48,co@pog.nz,"[Cycling, Dancing, Cooking]",8d162eb9-eefc-4b7e-bcad-6fb7a02b15b0,cb795afb-8dc3-482f-b3a0-14229a280afa
9,cc3e389d-be0d-467a-ba70-7c84f6504911,Myrtle,Little,36,saj@far.zm,"[Cooking, Cycling, Hiking, Rubgy, Bowling, Dan...",0d3a90d5-11bd-4f96-91ba-7e7ef80888f0,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f


In [13]:
# In the second option, first I match with age, then unwind by interest and finally match by interest.
pipeline = [
    {"$match": {'age': {'$lte': 50}}},
    {"$unwind": "$interests"},
    {"$match": {'interests': "Cooking"}},
]
cursor = db.employee.aggregate(pipeline)
pd.DataFrame(list(cursor))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,9f39da36-82cc-4353-ab90-d616105fa7c1,Emilie,Woods,40,ih@ri.ro,Cooking,b6c0b50a-d0e3-43bf-a2a4-8d4674c2a7e8,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,af27265e-6639-49f2-991e-193275a4111a,Thomas,Patterson,18,sug@gon.bf,Cooking,64fd714d-e219-4e45-888b-cc2238a8bd0b,5345fcb9-6297-4b9f-aa15-cbee8460f28f
2,00289d48-bad8-4b73-a359-a1a1f05c96e2,Sophia,Flores,22,ra@dupnejuk.nr,Cooking,8a430805-00b8-40a6-bd93-c950b544a83b,b12cd444-e65b-4bc2-8cf6-2dbe854a627b
3,da76e52b-b3db-4fc0-b0d6-435d1aed0cd9,Ollie,Barnett,25,ro@nemaw.et,Cooking,5d3eacc4-d1d8-459b-973c-3bc71feacf50,cb795afb-8dc3-482f-b3a0-14229a280afa
4,51643cd6-49bb-45d5-bd6e-717c62bb2869,James,Wilkins,27,hutfardu@vicbiri.gb,Cooking,3f10d9f7-57ef-40e1-a97d-d2ee53aa2c6e,cb795afb-8dc3-482f-b3a0-14229a280afa
5,f073a705-6546-4375-adb5-b224871776ef,Aaron,Carr,25,fekegim@lucul.tp,Cooking,f4e59d7e-ea40-442d-87be-106e3c46a554,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
6,457ef68c-9651-4925-bca0-15e246661d19,Alta,Sharp,34,jus@goal.bn,Cooking,5e97658e-5809-41b8-a088-eddbd81f86a7,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
7,840184a3-4c4d-4b15-8813-30fca6e7827b,Delia,Douglas,36,me@wak.ne,Cooking,456e18ae-c2f8-443e-899d-f2b893499695,b2a2844d-aeab-4602-b74c-01bf3b8e9c78
8,6157dc3b-ee2d-463a-b62b-1cd4eed7d575,Dominic,Wade,48,co@pog.nz,Cooking,8d162eb9-eefc-4b7e-bcad-6fb7a02b15b0,cb795afb-8dc3-482f-b3a0-14229a280afa
9,cc3e389d-be0d-467a-ba70-7c84f6504911,Myrtle,Little,36,saj@far.zm,Cooking,0d3a90d5-11bd-4f96-91ba-7e7ef80888f0,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f


#### Question 2  (10 Marks)

Insert a new Employee with the following properties:

* First Name: Jake 
* Last Name: Sample
* Email: jakesample@email.com
* Age: 26
* Interest: Biking, Hiking

Also, this employee works for 'Union Planters Corp' and lives at '573 Wojhas Square, Victoria'.
Verify that the insert succeeded and display the generated employees _id attribute.

__HINT__ An Employee document references a Workplace and Address document

In [17]:
# Define new element
new_employee = {
    "firstname": "Jake",
    "lastname": "Sample",
    "email": "jackesample@email.com",
    "age": 26,
    "interest": ["Biking", "Hiking"],
    "workplace_id": "5345fcb9-6297-4b9f-aa15-cbee8460f28f",
    "address_id": "9949fe3b-99ec-4485-b91d-823925db7d28"
}

In [18]:
# Inserted element
new_element = db.employee.insert_one(new_employee)

In [21]:
# Showing Id
new_element.inserted_id

ObjectId('6349aab690ae8e0ded2840a0')

In [23]:
# Showing whole docuemnt
pd.DataFrame(list(db.employee.find({"_id": new_element.inserted_id})))

,_id,firstname,lastname,email,age,interest,workplace_id,address_id
0,6349aab690ae8e0ded2840a0,Jake,Sample,jackesample@email.com,26,"[Biking, Hiking]",5345fcb9-6297-4b9f-aa15-cbee8460f28f,9949fe3b-99ec-4485-b91d-823925db7d28


#### Question 3 (10 Marks)

Delete all employees that work for 'Great Plains Energy Inc.' and are greater than 46 years old and likes 'Tennis'. Once you delete the employees verify the number of employees deleted.

In [26]:
# Checking the number of employees first
db.employee.estimated_document_count()

101

In [39]:
# Employees to delete
cursor = db.employee.find({"workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460",
                           "age": {"$gte": 46}
                           })
pd.DataFrame(list(cursor))

,_id,firstname,lastname,age,email,interests,address_id,workplace_id
0,153ba346-b33d-4611-802c-fd95c7cb078c,Adelaide,Curry,71,ov@sutec.mr,[Tennis],8264af1e-1c55-4ed8-96f9-e3b9e1eff7d5,a32bf18d-e0e5-48f2-a851-aa49c80f9460
1,34521596-d3e4-4b36-a60c-cab5ad425f45,Eugene,Mack,60,finho@zeuj.dm,"[Swimming, Bowling, Golf, Cricket]",7e73276a-27c8-4602-bc34-a1f44858ecb9,a32bf18d-e0e5-48f2-a851-aa49c80f9460
2,8e082f05-13b7-485d-89c4-259f1103f95c,Lester,Summers,48,wi@uvafej.pt,"[Bowling, Tennis]",62996de2-fcda-4894-95d5-fef0d11d92f4,a32bf18d-e0e5-48f2-a851-aa49c80f9460
3,25264147-ba59-4d78-ac7a-d1820b0dff0e,James,Becker,48,bozzona@vugrewa.ke,"[Hiking, Cricket, Rubgy]",79ad04ba-0967-4a65-986f-cc57ef6c3afd,a32bf18d-e0e5-48f2-a851-aa49c80f9460
4,dfce1d98-15f6-4b75-86f6-cd5d05cfdc04,Sadie,Park,80,vukhik@wejoce.nf,"[Golf, Fishing, Bowling, Dancing, Cooking, Ten...",2987cea4-b71d-4338-b0ef-a9b5feb3a138,a32bf18d-e0e5-48f2-a851-aa49c80f9460
5,86417f71-18cd-41c5-ad01-24b31ff2074f,Lucy,May,65,bi@udtaofe.tj,[Tennis],fe80c95a-a0e9-42c9-aa6a-56f27020842d,a32bf18d-e0e5-48f2-a851-aa49c80f9460


In [40]:
# Employees to delete
delete_result = db.employee.delete_many({"workplace_id": "a32bf18d-e0e5-48f2-a851-aa49c80f9460",
                           "age": {"$gte": 46}
                           })

In [41]:
print("Employees deleted:" + str(delete_result.deleted_count))

Employees deleted:6


In [42]:
# Checking the number of employees final
db.employee.estimated_document_count()

95

#### Question 4 (12 Marks)
Add a new field called 'industry' to all employees that work for 'Health Net Inc.' and populate the field with the value 'Health Care'.

__HINT__ Adding a new field to a document is like updating the document

In [50]:
# YOUR CODE HERE
update_result = db.employee.update_many(
    {"workplace_id": "a222385c-342c-43ea-adbc-9b487a2ee2be"},
    {"$set": {"industry": "Health Care"}}
)

In [51]:
print("Update Acknowledged:" + str(update_result.acknowledged))
print("Modified Count:" + str(update_result.modified_count))

Update Acknowledged:True
Modified Count:14


#### Question 5 (10 Marks)

Create an aggregate query to count the number of employees for each company and sort the output from largest employee count to lowest employee count.

__NOTE__ you will use a pipeline to achieve the computed result.  You should produce a result similar to the following table (the following table contains fake data)
<table>
    <tr><th></th><th>_id</th><th>count</th></tr>
    <tr><td>0</td><td>[Equity Residential Properties Trust]</td><td>19</td></tr>
    <tr><td>...</td><td>...</td><td>...</td></tr>
    <tr><td>7</td><td>[Bell Microproducts Inc.]</td><td>6</td></tr>
    <tr><td>8</td><td>[Kemet Corp.]</td><td>1</td></tr>
</table>

__HINT__ you should make use of the \\$lookup, \\$group and \\$sort pipeline operations

In [61]:
# In the second option, first I match with age, then unwind by interest and finally match by interest.
pipeline = [
    {"$unwind": "$workplace_id"},
    {"$group": {"_id": "$workplace_id", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
cursor = db.employee.aggregate(pipeline)
pd.DataFrame(list(cursor))

,_id,count
0,cb795afb-8dc3-482f-b3a0-14229a280afa,15
1,a222385c-342c-43ea-adbc-9b487a2ee2be,14
2,2b87eb84-e5b4-4f2c-9e13-dc3ba20a7f7f,13
3,b2a2844d-aeab-4602-b74c-01bf3b8e9c78,11
4,023acbac-b32a-477f-a65a-db11bd7d40f3,10
5,5345fcb9-6297-4b9f-aa15-cbee8460f28f,10
6,b12cd444-e65b-4bc2-8cf6-2dbe854a627b,7
7,50275ad1-8140-4e79-8818-21793e3eb0a3,6
8,b2903a4b-5688-4597-90ed-0f06d944bb6d,6
9,a32bf18d-e0e5-48f2-a851-aa49c80f9460,3


In [0]:
# YOUR CODE HERE

query_result = # YOUR CODE HERE

pd.DataFrame(list(query_result))